In [48]:
import pandas as pd

dataset = pd.read_parquet('data/train_data/big_dataset.parquet')
small_dataset = dataset.sample(2000)
small_dataset.to_parquet('data/train_data/small.parquet')

In [47]:
dataset['fps'][0]

[668,
 1147,
 1192,
 1405,
 1437,
 1590,
 1598,
 2458,
 2948,
 2974,
 3024,
 3153,
 3529,
 3955,
 4330,
 4644,
 4694,
 4812]